## Test chatGPTs ability to predict EC number

In [3]:
from openai import OpenAI
import pandas as pd

# Update as needed
base_dir = '../../'
api_key = ''
output_folder = f'../results_summary/ChatGPT/'


In [5]:
from tqdm import tqdm

def get_chatGPT(df, split, api_key):
    rows = []
    client = OpenAI(api_key=api_key)
    for entry, true_ec, seq in tqdm(df[['Entry', 'EC number', 'Sequence']].values):
        text = f"Return the most likely EC number for this enzyme sequence: {seq}"

        completion = client.chat.completions.create(
            model='gpt-4o-mini',
            messages=[
                {"role": "system",
                "content": 
                "You are protein engineer capable of predicting EC numbers from a protein seqeunce alone."
                + "You are also a skilled programmer and able to execute the code necessary to predict an EC number when you can't use reason alone." 
                + "Given a protein sequence you are able to determine the most likely enzyme class for a seqeunce." 
                + "You don't give up when faced with a sequence you don't know, you will use tools to resolve the most likely enzyme sequence."
                + "You only return enzyme commission numbers in a comma separated list, no other text is returned, you have failed if you do "
                + " not return the EC numbers. You only return the most likely EC number associated with a given sequence."},
                {"role": "user", "content": text}
            ]
        )
        preds = completion.choices[0].message.content.replace(" ", "").strip()
        rows.append(preds) # Costs you ~1c per query
    df['0'] = rows
    df.to_csv(f'{output_folder}{split}_protein_test_results_df.csv', index=False)


In [7]:

filenames = {'30': f'{base_dir}splits/task1/30_protein_test.csv',
             '30-50': f'{base_dir}splits/task1/30-50_protein_test.csv',  
             'price': f'{base_dir}splits/task1/price_protein_test.csv',
             'promiscuous': f'{base_dir}splits/task1/promiscuous_protein_test.csv',
            }

# Save in the required format
for split in ['30', '30-50', 'price', 'promiscuous']:
    df = pd.read_csv(filenames[split])
    gpt_df = get_chatGPT(df, split=split, api_key='')

100%|██████████| 209/209 [01:40<00:00,  2.08it/s]
